In [ ]:
import requests, base64
from dotenv import load_dotenv
import os
from fastapi.responses import RedirectResponse

load_dotenv()

# Load Spotify API credentials from .env file
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REDIRECT_URI = os.getenv("SPOTIFY_REDIRECT_URI")
SPOTIFY_SCOPES = os.getenv("SPOTIFY_SCOPES")


In [ ]:
# APPROACH 1: Simple Authorization URL Generation
# Generate Spotify authorization URL
# This is the first step in OAuth flow - redirect user to Spotify login
AUTH_URL = auth_url = (
        "https://accounts.spotify.com/authorize"
        f"?client_id={SPOTIFY_CLIENT_ID}"
        f"&response_type=code"
        f"&redirect_uri={SPOTIFY_REDIRECT_URI}"
        f"&scope={SPOTIFY_SCOPES}"
    )



In [ ]:
# Create redirect response object
redirect_auth = RedirectResponse(url=AUTH_URL)

In [5]:
print(redirect_auth)


In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
import requests, base64

# APPROACH 2: FastAPI-based OAuth Implementation
# FastAPI app with complete OAuth flow

app = FastAPI()

# Store credentials in variables for cleaner code
CLIENT_ID = SPOTIFY_CLIENT_ID
CLIENT_SECRET = SPOTIFY_CLIENT_SECRET
REDIRECT_URI = SPOTIFY_REDIRECT_URI
SCOPE = SPOTIFY_SCOPES

@app.get("/")
def login():
    """
    Endpoint to initiate Spotify login.
    Redirects user to Spotify authorization page.
    """
    auth_url = (
        "https://accounts.spotify.com/authorize"
        f"?client_id={CLIENT_ID}"
        f"&response_type=code"
        f"&redirect_uri={REDIRECT_URI}"
        f"&scope={SCOPE}"
    )
    return RedirectResponse(auth_url)

@app.get("/callback")
def callback(request: Request):
    """
    Callback endpoint after user authorizes on Spotify.
    Exchanges authorization code for access and refresh tokens.
    """
    # Extract authorization code from query parameters
    code = request.query_params.get("code")
    token_url = "https://accounts.spotify.com/api/token"

    # Create Base64 encoded authorization header
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
    headers = {"Authorization": f"Basic {auth_header}"}
    
    # Request tokens from Spotify
    data = {
        "grant_type": "authorization_code",
        "code": code,
        "redirect_uri": REDIRECT_URI
    }

    r = requests.post(token_url, headers=headers, data=data)
    token_info = r.json()
    # Extract tokens from response
    access_token = token_info.get("access_token")
    refresh_token = token_info.get("refresh_token")

    return {
        "access_token": access_token,
        "refresh_token": refresh_token
    }

@app.get("/me")
def get_user_profile(access_token: str):
    """
    Test endpoint to fetch current user's Spotify profile.
    Requires valid access_token as query parameter.
    """
    headers = {"Authorization": f"Bearer {access_token}"}
    r = requests.get("https://api.spotify.com/v1/me", headers=headers)
    return r.json()


In [ ]:
import requests
import base64
from urllib.parse import urlencode
from http.server import HTTPServer, BaseHTTPRequestHandler
import webbrowser
import time

# APPROACH 3: Local HTTP Server for Interactive Testing
# Complete OAuth flow with local server

CLIENT_ID = SPOTIFY_CLIENT_ID
CLIENT_SECRET = SPOTIFY_CLIENT_SECRET
REDIRECT_URI = SPOTIFY_REDIRECT_URI
SCOPE = SPOTIFY_SCOPES

# Step 1: Ask user to log in
params = {
    "client_id": CLIENT_ID,
    "response_type": "code",
    "redirect_uri": REDIRECT_URI,
    "scope": SCOPE,
}
auth_url = f"https://accounts.spotify.com/authorize?{urlencode(params)}"
print("Opening Spotify login URL...")
webbrowser.open(auth_url) # Opens browser to Spotify login page

# Step 2: Simple local HTTP server to catch redirect
code_container = {} # Shared container to store authorization code

class Handler(BaseHTTPRequestHandler):
    """
    Simple HTTP handler to catch the OAuth callback.
    Extracts the authorization code from the URL.
    """
    def do_GET(self):
        if "/callback" in self.path:
            from urllib.parse import parse_qs, urlparse
            # Parse authorization code from callback URL
            query = parse_qs(urlparse(self.path).query)
            code_container["code"] = query.get("code", [None])[0]
            # Send success response to browser
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"You can close this window now.")
        else:
            self.send_response(404)
            self.end_headers()

# Start server on localhost:8888 and wait for one request
server = HTTPServer(("localhost", 8888), Handler)
print("Waiting for Spotify authorization...")
server.handle_request() # Blocks until callback is received
server.server_close()

auth_code = code_container.get("code")
print("Authorization Code:", auth_code)

# Step 3: Exchange code for access + refresh token
token_url = "https://accounts.spotify.com/api/token"
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
headers = {"Authorization": f"Basic {auth_header}"}
data = {
    "grant_type": "authorization_code",
    "code": auth_code,
    "redirect_uri": REDIRECT_URI,
}

r = requests.post(token_url, headers=headers, data=data)
tokens = r.json()

# Extract token information
access_token = tokens.get("access_token")
refresh_token = tokens.get("refresh_token")
expires_in = tokens.get("expires_in")  # Token lifetime in seconds
expires_at = int(time.time()) + expires_in if expires_in else None # Calculate expiration timestamp

# Step 4: Example API call
# headers = {"Authorization": f"Bearer {tokens.get('access_token')}"}
# me = requests.get("https://api.spotify.com/v1/me", headers=headers).json()
# print(me)


Opening Spotify login URL...
Waiting for Spotify authorization...


127.0.0.1 - - [25/Oct/2025 13:28:02] "GET /callback?code=AQBRGQxwR9EacrEV4htGFrSrZD21WOcg9Af_F4U7OJ36NSMiHkFzFGS3zLAVaw-5gQ2lbp6uqTMSqFVoRiOIclZ7LYRH_B5P47-JB3l7_u0GHAtNEkxci-URQbghbnitJmACGMAL6ehEzNpyH6fEHQzQT7VGDLLBzLYA7oihnZTXChim1fsGDPGOTuRy5FTwzW8ixFA2GixV6IDbCkgxhHtlKFTJ4bac4oRarn8CVsfr0TTcQHT1dc4ZJh5hN0rNAuHXmxUpxE_wbGpvQIbnjr6EfHsZcw HTTP/1.1" 200 -


Authorization Code: AQBRGQxwR9EacrEV4htGFrSrZD21WOcg9Af_F4U7OJ36NSMiHkFzFGS3zLAVaw-5gQ2lbp6uqTMSqFVoRiOIclZ7LYRH_B5P47-JB3l7_u0GHAtNEkxci-URQbghbnitJmACGMAL6ehEzNpyH6fEHQzQT7VGDLLBzLYA7oihnZTXChim1fsGDPGOTuRy5FTwzW8ixFA2GixV6IDbCkgxhHtlKFTJ4bac4oRarn8CVsfr0TTcQHT1dc4ZJh5hN0rNAuHXmxUpxE_wbGpvQIbnjr6EfHsZcw
Access Token: BQAjCMHvqqTHX7c79D2-R1AHnW-udkLXCigOU3d_kKwgjbtWTYVhlN3WTbkFuTLfSXyNNWkv1Jtg4Wnb32-RtKc3IQ6jELK0Z5-DfVacOOR3OSPsCfeTqLN7gIjLgBhi4yfmMfhwIBjkVbjURiJzf_Z-j_9v3sn2LA_eIfsl1HbbwR_6uBP06rmIJ0MZB3O6KhpIISiHYmr2qmDjOO-qSWyndx4KufYFWxJ5uOO7dRNJf1YE1ieDVS00oN0YHcNn9ird52k95wwPMmErgMjGyw
Refresh Token: AQC5IbymrnPx4c-kYjSbARKNuPhJsfkcIWfaBemuSMJV0qHGhOYtOcIzxoxGytjjEd13MTvCcA-3LOhRBVuJ_CT3L8dwgvQhgqw67D0eT4q1eVbLaGeDEpFoiYRyAdstAnQ
{'display_name': 'draqa', 'external_urls': {'spotify': 'https://open.spotify.com/user/31zkynjlgmywa25xynvjao7vtcse'}, 'followers': {'href': None, 'total': 1}, 'href': 'https://api.spotify.com/v1/users/31zkynjlgmywa25xynvjao7vtcse', 'id': '31zkynjlgmywa2